In [2]:
import pandas as pd
import boto3
import json
import os
import psycopg2

In [ ]:
key = ''
secret = ''

ec2 = boto3.resource('ec2',
                    region_name='us-east-1',
                    aws_access_key_id=key,
                    aws_secret_access_key=secret)

s3 = boto3.resource('s3',
                    region_name='us-east-1',
                    aws_access_key_id=key,
                    aws_secret_access_key=secret)

iam = boto3.client('iam',
                    region_name='us-east-1',
                    aws_access_key_id=key,
                    aws_secret_access_key=secret)

redshift = boto3.client('redshift',
                    region_name='us-east-1',
                    aws_access_key_id=key,
                    aws_secret_access_key=secret)

In [1]:
%run create_tables.py

In [2]:
%run etl.py

loading 
    copy events from 's3://udacity-dend/log_data'    credentials 'aws_iam_role=arn:aws:iam::464175583074:role/dwhRole'    json 's3://udacity-dend/log_json_path.json'    region 'us-west-2'    timeformat as 'epochmillisecs'

loading 
    copy songs from 's3://udacity-dend/song_data'    credentials 'aws_iam_role=arn:aws:iam::464175583074:role/dwhRole'    json 'auto'    region 'us-west-2'
loading insert into songplay (songplay_id, start_time, user_id, level, song_id, artist_id,                                                     session_id, location, user_agent)
    select e.registration,
           e.ts::time as start_time,
           e.userId,
           e.level,
           s.song_id,
           s.artist_id,
           e.sessionId,
           e.location,
           e.userAgent
    from events e
    join songs s on (s.artist_name=e.artist)
    where e.page='NextSong';

loading insert into user1 (user_id, first_name, last_name, gender, level)
    select e.userId,
           e.firs